# Exploratory Data Analysis (EDA)

### Datathon challenge task
Predict the duration of time it takes for patients to receive metastatic cancer diagnosis.

### EDA Techniques
- Analysis
    - Load data
    - Understand the data:
        - Categorize the types of features
        - Data shape, types, missing (null) values, unique values
        - Categorical vs numeric data, and which of the categorical data is ordinal (has an order)
        - Statistical analysis
        - Visualizations
            - Univariate
            - Bivariate
            - Multivariate
    - Feature correlations
- Cleaning
    - Removing features
    - Handling missing data
    - Encoding categorical variables
- Feature engineering

In [ ]:
# Import libraries



# Analysis

## Load data

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Load .csv data as dataframes and make patient_id the index
df_train <- read_csv("data/train.csv", col_types = cols(.default = "i"))
df_test <- read_csv("data/test.csv", col_types = cols(.default = "i"))

# Look at the first few rows of data
head(df_train)

## Understand the data

### Categorize the types of features

Read the [data descriptions](https://www.kaggle.com/competitions/widsdatathon2024-challenge2/data) on Kaggle.

**Patient characteristics:** patient_race, payer_type, patient_age, patient_gender, bmi

**Patient location:** patient_state, patient_zip3, region, division

**Breast cancer diagnosis information:** breast_cancer_diagnosis_code, breast_cancer_diagnosis_desc, metastatic_cancer_diagnosis_code, 
    metastatic_first_novel_treatment, metastatic_first_novel_treatment_type

**Geo (zip-code level) demographic data:** Many! (population, income, education, rent, race, poverty etc)

**Climate data:** 72 columns showing the zip 3 Monthly Average Temperature for the patient’s zip 3 and month referenced

**Target variable:** metastatic_diagnosis_period

This code defines a function called initial_eda() that takes a dataframe as input. It then prints out the dimensions of the dataframe, the total number of missing values, the data type of each column, the number of unique values for each column, and the number of missing values for each column.

Note that the syntax for some functions may be slightly different between Python and R. However, the general idea remains the same.

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Define a function that shows the data dimensions (# rows and columns), total # NA values, and data types, 
# number of distinct NA (null) values for each column (feature)
initial_eda <- function(df){
  if (class(df) != "data.frame"){
    stop("Expect a dataframe")
  }
  cat("Dimensions : ", nrow(df), "rows, ", ncol(df), "columns\n")
  cat("Total NA Values : ", sum(is.na(df)), "\n")
  cat(paste0(sprintf("%38s %10s     %10s %10s\n", "Column Name", "Data Type", "#Distinct", "NaN Values")))
  col_name <- names(df)
  dtyp <- sapply(df, class)
  uniq <- apply(df, 2, function(x) length(unique(x)))
  na_val <- apply(df, 2, function(x) sum(is.na(x)))
  for (i in seq_along(col_name)){
    cat(paste0(sprintf("%38s %10s   %10s %10s\n", col_name[i], dtyp[i], uniq[i], na_val[i])))
  }
}

In [ ]:
# Call the function on our training data

initial_eda(df_train)

In [ ]:
# Call the function on our testing data

initial_eda(df_test)

#### Observations

**Data types**

- 11 object types in train and test - these are categorical features
    - None of these seem to be ordinal

- 3 integer types in train, 2 in test - zip, age, and the target variable (metastatic_diagnosis_period)

- The rest of the features are floats

**# distinct values**

- Only 1 distinct value for patient_gender (female) in train and test data, so we can drop this feature

- Only 1 distinct value for metastatic_first_novel_treatment_type in train and test data, and only 11 rows contain a value in train and 7 in test, so we can drop this feature

- Only 2 distinct values for metastatic_first_novel_treatment in train and test data, and only 11 rows contain a value in train and 7 in test, so we can probably drop this feature, but look for any strong correlation with the target feature

- patient_zip3 and population have the same number of unique values in train (751) and test (669), so do we need them both? 
Population is a size, so it might be important. Is zip random? How strong is the correlation with the target variable?

**NaN values - need to start thinking about how to handle these**

- patient_race: ~half are missing in train and test

- payer_type: 13-14% are missing in train and test

- bmi: 69-70% are missing in train and test

- metastatic_cancer_diagnosis_code and metastatic_first_novel_treatment_type: almost all are missing

- Several features (23) are missing 5 values in the train data but none of those are missing in the test data. Are the 5 missing values for
each feature in the same row? If so, can we just drop those rows? (Yes, all are in the same rows and they are in the same zip, and those
are the only 5 records for that zip, but that zip is not in the test data, so we can probably drop them.)

- Avg temps: Missing 0-180 in train and 0-95 in test. The 180/95 are in Apr-14 and are all in NY (6 zips) or ID (1 zip) in both train
and test

### Drop features and observations that logically add no value

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Drop features

# Only one value for patient_gender
# Only one value for metastatic_first_novel_treatment_type and two values for metastatic_first_novel_treatment, 
# and there are only 11 non-null rows in the train data, and 7 non-null in the test data

drop_features <- c('metastatic_first_novel_treatment',         # rarely filled in
                  'metastatic_first_novel_treatment_type',    # rarely filled in
                  'patient_gender')                           # always the same value

# Always perform the same actions on train and test data
df_train <- df_train %>% select(-one_of(drop_features))
df_test <- df_test %>% select(-one_of(drop_features))

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Drop the 5 rows that have 23 missing features all from the same zip; that zip is not present in the test set
df_train <- df_train %>% filter(patient_zip3 != 772)

# Check the shape (originally we had 13173 rows)
dim(df_train)

### Statistical summary

In [ ]:
# this doesn't look right

# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Do not truncate
options(repr.plot.width=16, repr.plot.height=8)

#### Observations

**patient_zip3** max in test is different than train so we know there are values in test that are not in train (and probably vice versa)

**patient_age** range from 18-91

**bmi** max is 97 in train but only 43.7 in test! Looking closer, there is one 90 and one 97 in the train data, which are anomalies,
perhaps we remove those rows

**target variable** min=0, max=365, mean=96.5, std=109, count of 0=3126, count >= 350=224

### Categorical vs numerical features

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Create variables for the categorical and numeric features of df_train (df_test will be the same except the target col)
cat_cols <- names(df_train)[sapply(df_train, is.factor)]
num_cols <- names(df_train)[sapply(df_train, is.numeric)]
target_col <- "metastatic_diagnosis_period"

# Print the list of categorical and numerical features
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

### Compare unique values in df_train and df_test

Which features are present in the training data but not in the test data, and vice versa

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Iterate over the columns of df_train
for (col in names(df_train)) {
  # Check if the column exists in df_test
  if (col %in% names(df_test)) {
    # Get the unique values of col in df_train
    unique_values_train <- unique(df_train[[col]])
    # Get the unique values of col in df_test
    unique_values_test <- unique(df_test[[col]])
    # Find the differences between the two sets
    differences <- setdiff(unique_values_train, unique_values_test)
    diff_count <- length(differences)
    if (length(differences) > 0) {
      cat(paste0(col, " has the following unique values that are not present in the test dataset: ", differences, "\n"))
      cat(paste0(col, " has the following number of differences: ", diff_count, "\n"))
    }
  }
}

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Iterate over the columns of df_test
for (col in names(df_test)) {
  # Check if the column exists in df_train
  if (col %in% names(df_train)) {
    # Get the unique values of col in df_test
    unique_values_test <- unique(df_test[[col]])
    # Get the unique values of col in df_train
    unique_values_train <- unique(df_train[[col]])
    # Find the differences between the two sets
    differences <- setdiff(unique_values_test, unique_values_train)
    diff_count <- length(differences)
    if (length(differences) > 0) {
      cat(paste0(col, " has the following unique values that are not present in the train dataset: ", differences, "\n"))
      cat(paste0(col, " has the following number of differences: ", diff_count, "\n"))
    }
  }
}

#### Observations

**patient_race, payer_type, patient_state, Region, Division, patient_age** Same values in train and test

**patient_zip3 and population** Train has 93 zips/populations not in test, test has 12 zips/populations not in train

**breast_cancer_diagnosis_code** Train has 7 values not in test (but just 1 or a few occurences of each), test has 1 value not in train (but only 1 occurrence of it)

**metastatic_cancer_diagnosis_code** Train has 7 values not in test (but just 1 or a few occurences of each), test has 2 values not in train (but only 1 occurrence of each)

### Visualizations

Categorical variables can be visualized using a Count plot, Bar Chart, Pie Plot, etc.

Numerical Variables can be visualized using Histogram, Box Plot, Density Plot, etc.

#### Univariate analysis

Looks at each feature individually

In [ ]:
# Histogram and box plot of numerical data for df_train

for (col in num_cols) {
  cat(col, "\n")
  cat("Skew :", round(skew(df_train[[col]]), 2), "\n")
  hist(df_train[[col]], main=col, xlab=col, ylab="Frequency", col="lightblue")
  boxplot(df_train[[col]], main=col, xlab=col, col="lightblue")
  cat("\n")
}

In [ ]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
# Histogram and box plot of numerical data for df_test

for (col in num_cols) {
  cat(col, "\n")
  cat("Skew :", round(skew(df_test[[col]]), 2), "\n")
  hist(df_test[[col]], main=col, xlab=col, ylab="Frequency", col="lightblue")
  boxplot(df_test[[col]], main=col, xlab=col, col="lightblue")
  cat("\n")
}

#### Observations of data

**bmi** 3 major outliers for train data, perhaps we drop those rows

**density** is quite skewed and has many outliers, but same for both data sets

**metastatic_diagnosis_period** Median is less than 50 days, has a right positive skew

**age_over_80** Out of proportion between the data sets based on histograms (box plots are very similar)

In [ ]:
# Count plot of categorical data for df_train

library(ggplot2)

fig <- ggplot(data = df_train) + 
  geom_bar(mapping = aes(x = patient_race)) + 
  labs(title = "Bar plot for patient_race variable", 
       subtitle = "Count of patients by race", 
       x = "Race", 
       y = "Count") + 
  theme(plot.title = element_text(hjust = 0.5))

fig

In [ ]:
# Count plot of categorical data for df_test

library(ggplot2)

fig <- ggplot(data = df_test) + 
  geom_bar(mapping = aes(x = patient_race)) + 
  labs(title = "Bar plot for patient_race variable", 
       subtitle = "Count of patients by race", 
       x = "Race", 
       y = "Count") + 
  theme(plot.title = element_text(hjust = 0.5))

fig

In [ ]:
# Group the data by breast_cancer_diagnosis_code and calculate the mean metastatic_diagnosis_period

grouped_data <- aggregate(df_train$metastatic_diagnosis_period, by = list(df_train$breast_cancer_diagnosis_code), FUN = mean)
head(grouped_data)

In [ ]:
# Plot a bar chart of the mean metastatic_diagnosis_period for each breast_cancer_diagnosis_code

barplot(grouped_data$x, main="Mean Metastatic Diagnosis Period for Each Breast Cancer Diagnosis Code", xlab="Diagnosis Code", ylab="Mean Metastatic Diagnosis Period", names.arg=grouped_data$Group.1)

In [ ]:
# Group the data by metastatic_cancer_diagnosis_code and calculate the mean metastatic_diagnosis_period

grouped_data <- aggregate(df_train$metastatic_diagnosis_period, by=list(df_train$metastatic_cancer_diagnosis_code), FUN=mean)
head(grouped_data)

In [ ]:
# Plot a bar chart of the mean metastatic_diagnosis_period for each metastatic_cancer_diagnosis_code

barplot(grouped_data$x, names.arg=grouped_data$Group.1, main="Mean Metastatic Diagnosis Period for Each Metastatic Cancer Diagnosis Code", xlab="Diagnosis Code", ylab="Mean Metastatic Diagnosis Period")

## Visualizations - bivariate analysis

In [ ]:
# Cannot convert to R

## Clean data

In [ ]:
# Data prep - replace categorical column nulls with 'unknown'

for (column in cat_cols) {
  df_train[[column]] <- replace(df_train[[column]], is.na(df_train[[column]]), "unknown")
  df_test[[column]] <- replace(df_test[[column]], is.na(df_test[[column]]), "unknown")
}

In [ ]:
# Drop target from num_cols

num_cols <- setdiff(num_cols, target_col)

In [ ]:
# Data prep - remaining nulls are in numeric columns - convert them to mean values from df_train
# (since df_train has more data and will likely have a more representative mean value than df_test)

for (column in num_cols) {
  mean_value <- mean(df_train[[column]], na.rm = TRUE)
  df_train[[column]] <- ifelse(is.na(df_train[[column]]), mean_value, df_train[[column]])
  df_test[[column]] <- ifelse(is.na(df_test[[column]]), mean_value, df_test[[column]])
}

head(df_train)

In [ ]:
# Convert categorical columns to numbers using LabelEncoder()

for (column in cat_cols) {
  combined_data <- rbind(df_train, df_test)
  le <- LabelEncoder()
  le$fit(combined_data[[column]])
  df_train[[column]] <- le$transform(df_train[[column]])
  df_test[[column]] <- le$transform(df_test[[column]])
}

head(df_train)

## Correlations

### Correlation EDA by feature category

Since there are so many features, a heatmap of all features is not comprehensible. So let us break down the features in a way that makes sense for the analysis.

In [ ]:
patient_demographics <- c("patient_race",
                          "patient_state",
                          "patient_zip3",
                          "patient_age",
                          "bmi",
                          "metastatic_diagnosis_period")

diagnosis_codes <- c("breast_cancer_diagnosis_code",
                     "metastatic_cancer_diagnosis_code")

treatment <- c("metastatic_first_novel_treatment",
               "metastatic_first_novel_treatment_type")

demographics_by_zip_code <- c("population",
                              "density",
                              "age_median",
                              "male",
                              "female",
                              "married",
                              "family_size",
                              "income_household_median",
                              "income_household_six_figure",
                              "home_ownership",
                              "housing_units",
                              "home_value",
                              "rent_median",
                              "education_college_or_above",
                              "labor_force_participation",
                              "unemployment_rate",
                            "metastatic_diagnosis_period")
race_and_ethnicity <- c("race_white",
                        "race_black",
                        "race_asian",
                        "race_native",
                        "race_pacific",
                        "race_other",
                        "race_multiple",
                        "hispanic",
                        "metastatic_diagnosis_period")

age_groups <- c("age_under_10",
                "age_10_to_19",
                "age_20s",
                "age_30s",
                "age_40s",
                "age_50s",
                "age_60s",
                "age_70s",
                "age_over_80", 
                "metastatic_diagnosis_period")

marital_status <- c("divorced",
                    "never_married",
                    "widowed",
                    "metastatic_diagnosis_period")
income <- c("family_dual_income",  # Create this feature based on family_dual_income
            "income_household_under_5",
            "income_household_5_to_10",
            "income_household_10_to_15", 
            "income_household_15_to_20",
            "income_household_20_to_25",
            "income_household_25_to_35",
            "income_household_35_to_50",
            "income_household_50_to_75",
            "income_household_75_to_100",
            "income_household_100_to_150",
            "income_household_150_over", 
            "income_individual_median",
            "metastatic_diagnosis_period")

socioeconomic_factors <- c("poverty",
                           "rent_burden",
                           "metastatic_diagnosis_period")

education <- c("education_less_highschool",
               "education_highschool",
               "education_some_college",
               "education_bachelors",
               "education_graduate",
               "education_stem_degree",
               "metastatic_diagnosis_period")

employment <- c("self_employed",
                "farmer",
                "metastatic_diagnosis_period")

other <- c("disabled",
           "limited_english",
           "commute_time",
           "health_uninsured",
           "veteran",
           "metastatic_diagnosis_period")

# Time-based averages (needs further handling based on your approach)
time_based_averages <- c('Average of Jan-13', 'Average of Feb-13', 'Average of Mar-13', 'Average of Apr-13', 'Average of May-13', 'Average of Jun-13', 'Average of Jul-13', 'Average of Aug-13', 'Average of Sep-13', 'Average of Oct-13', 'Average of Nov-13', 'Average of Dec-13', 'Average of Jan-14', 'Average of Feb-14', 'Average of Mar-14', 'Average of Apr-14', 'Average of May-14', 'Average of Jun-14', 'Average of Jul-14', 'Average of Aug-14', 'Average of Sep-14', 'Average of Oct-14', 'Average of Nov-14', 'Average of Dec-14', 'Average of Jan-15', 'Average of Feb-15', 'Average of Mar-15', 'Average of Apr-15', 'Average of May-15', 'Average of Jun-15', 'Average of Jul-15', 'Average of Aug-15', 'Average of Sep-15', 'Average of Oct-15', 'Average of Nov-15', 'Average of Dec-15', 'Average of Jan-16', 'Average of Feb-16', 'Average of Mar-16', 'Average of Apr-16', 'Average of May-16', 'Average of Jun-16', 'Average of Jul-16', 'Average of Aug-16', 'Average of Sep-16', 'Average of Oct-16', 'Average of Nov-16', 'Average of Dec-16', 'Average of Jan-17', 'Average of Feb-17', 'Average of Mar-17', 'Average of Apr-17', 'Average of May-17', 'Average of Jun-17', 'Average of Jul-17', 'Average of Aug-17', 'Average of Sep-17', 'Average of Oct-17', 'Average of Nov-17', 'Average of Dec-17', 'Average of Jan-18', 'Average of Feb-18', 'Average of Mar-18', 'Average of Apr-18', 'Average of May-18', 'Average of Jun-18', 'Average of Jul-18', 'Average of Aug-18', 'Average of Sep-18', 'Average of Oct-18', 'Average of Nov-18', 'Average of Dec-18')

#### Patient Demographics

In [ ]:
# Load libraries
library(tidyverse)
library(ggcorrplot)

# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, patient_demographics])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

####  Demographics by Zip Code

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, demographics_by_zip_code])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

#### race_and_ethnicity 

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, race_and_ethnicity])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

#### Age group correlation 

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, age_groups])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

####   marital_status

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, marital_status])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

### Income Correlation

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, income])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

#### Socio-economic factors

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, socioeconomic_factors])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

#### Education

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, education])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

#### Employment

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, employment])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

#### Other

In [ ]:
# Calculate correlation coefficients
correlation_matrix <- cor(df_train[, other])

# Create a heatmap
ggcorrplot(correlation_matrix, label = TRUE, type = "lower", 
           title = "Correlation Matrix (Features vs. Target)", 
           lab_size = 3, ggtheme = ggplot2::theme_minimal())

### Correlation between each features and the target using .corr() with spearman and pearson methods

In [ ]:
# Calculate Spearman correlation
df_train %>%
  cor(method = "spearman") %>%
  .$target_col %>%
  abs() %>%
  sort(decreasing = TRUE)

# Regression modeling